In [1]:
import pandas as pd
import utils.fetcher_utils as fetcher
import utils.pipeline_util as pu
import utils.transformer_util as tu
from utils.prediction_builder import PredictionDFBuilder
import numpy as np
import IMDb_Predictor_GUI_II as gui
import tkinter as tk
from tkinter import ttk

In [2]:
%gui tk

In [3]:
# set pandas options
pd.set_option('display.max_columns', None)

In [4]:
# fetch the data 
df = fetcher.aquireIMDbDataFrame()
df.head()

File path: ../resources/movie_metadata.csv
Reading data from ../resources/movie_metadata.csv


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [5]:
# preprocess the data
# processed_df = preproc.preprocess_data(df)
# processed_df.head()
model, best_y_pred, preprocessed_df = pu.run_pipeline(df, True)

Training model
Creating train and test splits
Metrics for Linear Regression: {'Mean Squared Error': 0.904988243075293, 'R-squared': 0.14680014339946434, 'Adjusted R-squared': 0.11069293943674574}
Training model
Creating train and test splits
Metrics for Lasso Regression: {'Mean Squared Error': 0.9068641059845162, 'R-squared': 0.14503162764536703, 'Adjusted R-squared': 0.10884958053944993}
Training model
Creating train and test splits
Metrics for Ridge Regression: {'Mean Squared Error': 0.9049881994516595, 'R-squared': 0.1468001845267145, 'Adjusted R-squared': 0.11069298230449076}
Training model
Creating train and test splits
Metrics for Support Vector Machine: {'Mean Squared Error': 0.8975344497888188, 'R-squared': 0.15382738978836663, 'Adjusted R-squared': 0.11801757713050753}
The best model is Support Vector Machine with an adjusted R-squared of 0.11801757713050753


In [6]:
#pd.Series(y_pred).describe()

In [7]:
builder = PredictionDFBuilder(df)
prediction_df = (
    builder
    .add_actor_1("Clint Eastwood")
    .add_actor_2("Meryl Streep")
    .add_actor_3("Tom Hanks")
    .add_director("Gore Verbinski")
    .add_rating("PG-13")
    .add_genre("Thriller")
    .build()
)

prediction_df


Adding actor 1: Clint Eastwood
Adding actor 2: Meryl Streep
Adding actor 3: Tom Hanks
Adding director: Gore Verbinski
Adding rating: PG-13
Adding genre: Thriller
   Action  Adventure  Comedy  Crime  Drama  Family  Fantasy  Horror  Romance  \
0       0          0       0      0      0       0        0       0        0   

   Sci-Fi  Thriller  actor_total_facebook_likes        budget  \
0       0         1                       27000  3.975262e+07   

   cast_total_facebook_likes  director_facebook_likes  director_frequency  \
0                       9699                      563                 7.0   

   duration  facenumber_in_poster         gross  movie_facebook_likes  \
0       107                     1  4.846841e+07                  7525   

   num_critic_for_reviews  num_user_for_reviews  num_voted_users  other_genre  \
0                     140                   272            83668            0   

   rating_bin  title_year  total_actor_frequency  
0           0        2024     

,Action,Adventure,Comedy,Crime,Drama,Family,Fantasy,Horror,Romance,Sci-Fi,Thriller,actor_total_facebook_likes,budget,cast_total_facebook_likes,director_facebook_likes,director_frequency,duration,facenumber_in_poster,gross,movie_facebook_likes,num_critic_for_reviews,num_user_for_reviews,num_voted_users,other_genre,rating_bin,title_year,total_actor_frequency
0,0,0,0,0,0,0,0,0,0,0,1,27000,3.975262e+07,9699,563,7.0,107,1,4.846841e+07,7525,140,272,83668,0,0,2024,2855.0


In [8]:
prediction = model.predict(prediction_df)
percentile = np.percentile(prediction, 50)
print(f"Predicted IMDb Score: {prediction[0]} {percentile}")
prediction

Predicted IMDb Score: 6.307430157135122 6.307430157135122


array([6.30743016])

In [ ]:
root = tk.Tk()
gui = gui.PredictionDFBuilderGUI(root, df, model)
root.mainloop()


2024-11-30 16:40:19.109 python[5964:68279] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-30 16:40:19.110 python[5964:68279] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Adding actor 1: Johnny Depp
Adding actor 2: Orlando Bloom
Adding actor 3: Stephanie Sigman
Adding director: Gore Verbinski
Adding rating: PG-13
Adding genre: Comedy


Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/Users/brianhansen-turton/UPENN_Code/module-16-project/src/IMDb_Predictor_GUI_II.py", line 78, in build_prediction_df
    .build()
     ^^^^^^^
  File "/Users/brianhansen-turton/UPENN_Code/module-16-project/src/utils/prediction_builder.py", line 185, in build
    self.__calculate_actor_facebook_likes()
  File "/Users/brianhansen-turton/UPENN_Code/module-16-project/src/utils/prediction_builder.py", line 114, in __calculate_actor_facebook_likes
    self.__actor_rating(self.actor_1_name, 'actor_1_name', 'actor_1_facebook_likes')
  File "/Users/brianhansen-turton/UPENN_Code/module-16-project/src/utils/prediction_builder.py", line 128, in __actor_rating
    row = self.dataframe[self.dataframe [name_col] == name]
                         ~~~~~~~~~~~~~~~^^^^^^^^^^
  File "/opt/anacond